In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number,get_begin_index
import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from sample.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from sample.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children
from collections import Counter


In [2]:
directory = directory_project

plate_number = 102
plate = get_postion_number(plate_number)
begin = 262
end = 287
dates_datetime = get_dates_datetime(directory,plate)
dates = dates_datetime[begin:end+1]
exp = Experiment(plate,directory)
exp.load(dates) #for method 2

In [3]:
plate

40

In [4]:
exp = get_exp((40,262,287),directory_project)


begin = 2020-12-26 20:04:00 
  end = 2020-12-30 23:33:00


In [3]:
experiment = exp
exclude_bottom_factor = 0.98
tips = [
        node
        for node in experiment.nodes
        if node.degree(node.ts()[0]) == 1
        and node.pos(node.ts()[0])[0]
        <= experiment.boundaries_x[1] * exclude_bottom_factor
    ]
problems2 = []
problems = []

hyphaes = []
select = {}
    #     for i in range(20):
for i, tip in enumerate(tips):
    if i % 200 == 0:
        print(i / len(tips))
    #         tip = choice(tips)
    hyphae = Hyphae(tip)
    roots = []
    for t in tip.ts():
        #             print(t,tip)
        if tip.degree(t) == 1:
            root, edges, nodes = hyphae.get_edges(t, 200)
            roots.append(root)
    occurence_count = Counter(roots)
    if len(occurence_count.values()) >= 2:
        select[tip]=occurence_count
    if (
        len(occurence_count.values()) >= 2
        and occurence_count.most_common(2)[0][0] != roots[0]
        and occurence_count.most_common(2)[1][1]
        / occurence_count.most_common(2)[0][1]
        >= 0.75
    ):
        problems2.append(tip)
    else:
        hyphae.root = occurence_count.most_common(2)[0][0]
        if hyphae.root != roots[0]:
            problems.append((tip,roots[0]))
        hyphae.ts = sorted(set(hyphae.ts).intersection(set(hyphae.root.ts())))
        hyphaes.append(hyphae)

0.0
0.02884338044418806
0.05768676088837612
0.08653014133256418
0.11537352177675224
0.1442169022209403
0.17306028266512835
0.2019036631093164
0.23074704355350448
0.25959042399769255
0.2884338044418806
0.31727718488606865
0.3461205653302567
0.37496394577444475
0.4038073262186328
0.4326507066628209
0.46149408710700895
0.490337467551197
0.5191808479953851
0.5480242284395731
0.5768676088837612
0.6057109893279492
0.6345543697721373
0.6633977502163253
0.6922411306605134
0.7210845111047015
0.7499278915488895
0.7787712719930776
0.8076146524372656
0.8364580328814537
0.8653014133256418
0.8941447937698298
0.9229881742140179
0.9518315546582059
0.980674935102394


In [4]:
len(tips)

6934

In [46]:
tips = select.keys()
tip  = choice(list(tips))

In [47]:
tip,select[tip]

(Node(233),
 Counter({Node(230): 2,
          Node(229): 1,
          Node(559): 1,
          Node(663): 11,
          Node(4813): 2,
          Node(6883): 3,
          Node(10741): 2,
          Node(13060): 2}))

In [48]:
plt.close('all')
plot_raw_plus(exp,tip.ts()[0],[tip.label]+[node.label for node in select[tip].keys()])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
exp.load_compressed_skel()

KeyboardInterrupt: 

In [27]:
RH, BAS, max_speeds, total_growths, widths, lengths, branch_frequ,select_hyph = get_rh_bas(exp,criter)

In [10]:
list_node = [rh.root.label for rh in RH]
# [rh.end.label for rh in RH]+

In [101]:
plt.close('all')
# plot_raw_plus(exp,0,[node.label for node in exp.nodes if node.is_in(0) and node.degree(0)==3 ])
# plot_raw_plus(exp,-10,[node.label for node in roots_new_branch])
plot_raw_plus(exp,-10,[node[0].label for node in problems])

TypeError: 'Node' object is not subscriptable

In [72]:
plt.close('all')
couple = choice(problems)
tip,root_init = couple
root = [hyph.root.label for hyph in hyphaes if hyph.end.label == tip.label][0]
plot_raw_plus(exp,Node(root,exp).ts()[-1],[tip.label,root,root_init.label])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
error_nodes = [node for node in exp.nodes if node.degree(node.ts()[0])>=3 and node.ts()[-1]!=exp.ts-1]

In [5]:
node_problem = choice(error_nodes)
node_problem.ts()

[28, 29, 30]

In [91]:
time_problem = [node.ts()[-1] for node in error_nodes]

In [15]:
count_time_error = [len([node for node in error_nodes if t in node.ts()]) for t in range(exp.ts)]
count_time_total = [len([node for node in exp.nodes if t in node.ts()]) for t in range(exp.ts)]

In [16]:
np.array(count_time_error)/np.array(count_time_total)

array([0.26446281, 0.2585034 , 0.25786164, 0.24637681, 0.23012552,
       0.2265625 , 0.20364742, 0.21247113, 0.20450607, 0.23142857,
       0.24330617, 0.23660263, 0.22421525, 0.20792079, 0.2158167 ,
       0.20122783, 0.20214782, 0.20463768, 0.18625277, 0.1933162 ,
       0.19531996, 0.18902992, 0.1783247 , 0.18185452, 0.17192722,
       0.157173  , 0.15560795, 0.15049004, 0.14975139, 0.14348079,
       0.13163972, 0.12022472, 0.12012729, 0.11036872, 0.09257983,
       0.08050759, 0.04968589, 0.        ])

In [17]:
count_time_error

[32,
 38,
 41,
 51,
 55,
 58,
 67,
 92,
 118,
 162,
 209,
 234,
 250,
 252,
 292,
 295,
 320,
 353,
 336,
 376,
 409,
 417,
 413,
 455,
 463,
 447,
 462,
 476,
 512,
 493,
 456,
 428,
 453,
 446,
 403,
 387,
 261,
 0]

In [92]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(time_problem)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 17.,  32.,  77., 207., 200., 330., 474., 421., 637., 913.]),
 array([ 0. ,  3.6,  7.2, 10.8, 14.4, 18. , 21.6, 25.2, 28.8, 32.4, 36. ]),
 <a list of 10 Patch objects>)

In [89]:
len(error_nodes)/len(exp.nodes)

0.2840948127791137

In [28]:
roots_new_branch = []
total_growth = 0
lapse = 2
for rh in RH:
    for index in range(len(rh.ts[:-lapse])):
        t = rh.ts[index]
        tp1 = rh.ts[index + lapse]
        pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
        roots_new_branch+=nodes[1:-1] 
        total_growth += np.sum([get_length_um(seg) for seg in pixels])
len(roots_new_branch)

767

In [29]:
total_growth/len(roots_new_branch)

1050.9270230184475

In [25]:
total_growth/len(roots_new_branch)

741.6296301130968

In [14]:
total_growth/len(roots_new_branch)

1303.7193986115556

In [21]:
plt.close('all')
exp.plot([0],[[node.label for node in exp.nodes if node.is_in(0) and node.degree(0)==3 ]])

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
def plot_raw_plus(exp,t0,node_list,shift=(0,0),compress=5):
    date = exp.dates[t0]
    directory_name = get_dirname(date,exp.plate)
    path_snap = exp.directory + directory_name
    skel = read_mat(path_snap + "/Analysis/skeleton_pruned_realigned.mat")
    Rot = skel["R"]
    trans = skel["t"]
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    fig = plt.figure()
    size = 5
    ax = fig.add_subplot(111)
    grey = 1
    ax.imshow(im)
    bbox = dict(boxstyle="circle", fc=colors.rgb2hex((grey, grey, grey)))
    #             ax.text(right, top, time,
    #                 horizontalalignment='right',
    #                 verticalalignment='bottom',
    #                 transform=ax.transAxes,color='white')
    for node in node_list:
        #                     print(self.positions[ts[i]])
        if node in exp.positions[t0].keys():
            xs,ys = exp.positions[t0][node]
            rottrans = np.dot(np.linalg.inv(Rot), np.array([xs, ys] - trans))
            ys, xs = round(rottrans[0]), round(rottrans[1])
            t = ax.text(
                (xs - shift[1]) // compress,
                (ys - shift[0]) // compress,
                str(node),
                ha="center",
                va="center",
                size=size,
                bbox=bbox,
            )
    plt.show()

In [6]:
# plt.close('all')
exp.plot_raw(0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
a=[11477,13117,13]
b=[12908,9441,11]

In [22]:
x=b
list_node = nx.shortest_path(
            exp.nx_graph[0],
            source=x[0],
            target=x[1],
            weight="weight",
        )

In [23]:
get_length_um_node_list(list_node,exp,0)/x[2]

1937.5543062006407

In [ ]:
num_branch = 13

In [ ]:
11465